In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import time
from zipfile import ZipFile
import warnings
import pickle
import torch
from torch.utils.data import Dataset
import tsfel
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import lightgbm as lgb
from hiclass import LocalClassifierPerNode, LocalClassifierPerParentNode, LocalClassifierPerLevel
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
warnings.filterwarnings('ignore')

# Split by slope

In [2]:
trn_base = pd.read_csv("../downloads/train_data_features_v3_fixed/train_features_full_v3.csv")
tst_base = pd.read_csv("../downloads/test_features_full_v3.csv")

In [44]:
trn_zs_idx = list(trn_base[trn_base['0_Slope'] == 0].index)
trn_nz_idx = list(trn_base[trn_base['0_Slope'] != 0].index)

tst_zs_idx = list(tst_base[tst_base['0_Slope'] == 0].index)
tst_nz_idx = list(tst_base[tst_base['0_Slope'] != 0].index)

# Prepare features

In [45]:
pre_feature_list = ['0_Absolute energy', '0_Area under the curve', '0_Autocorrelation', '0_Average power', '0_Centroid', '0_ECDF Percentile Count_0', '0_ECDF Percentile Count_1', '0_ECDF Percentile_0', '0_ECDF Percentile_1', '0_ECDF_0', '0_ECDF_1', '0_ECDF_2', '0_ECDF_3', '0_ECDF_4', '0_ECDF_5', '0_ECDF_6', '0_ECDF_7', '0_ECDF_8', '0_ECDF_9', '0_Entropy', '0_Histogram mode', '0_Interquartile range', '0_Kurtosis', '0_Max', '0_Mean', '0_Mean absolute deviation', '0_Mean absolute diff', '0_Mean diff', '0_Median', '0_Median absolute deviation', '0_Median absolute diff', '0_Median diff', '0_Min', '0_Negative turning points', '0_Neighbourhood peaks', '0_Peak to peak distance', '0_Positive turning points', '0_Root mean square', '0_Signal distance', '0_Skewness', '0_Slope', '0_Standard deviation', '0_Sum absolute diff', '0_Variance', '0_Zero crossing rate', '0_Fundamental frequency', '0_Human range energy', '0_Max power spectrum', '0_Maximum frequency', '0_Median frequency', '0_Power bandwidth', '0_Wavelet entropy', 'value_median', 'value_mean', 'value_qmean', 'value_max', 'value_min', 'value_maxmin', 'value_diffmax', 'value_diffmin', 'value_diffmean', 'value_diffqmean', 'value_diffmedian', 'value_diffmaxmin', 'time_diffmean', 'time_diffqmean', 'time_diffmax', 'time_diffmin', 'time_diffmedian', 'value_std', 'value_var', 'value_diffstd', 'value_diffvar', 'time_diffstd', 'time_diffvar', 'time_burstiness', 'time_total', 'time_event_density', 'time_entropy', 'time_slope']
zero_slope_feat_cols = ['0_Signal distance', '0_Maximum frequency', '0_Median frequency', '0_Power bandwidth', '0_Spectral centroid', '0_Spectral distance', '0_Spectral kurtosis', '0_Spectral positive turning points', '0_Spectral roll-off', '0_Spectral spread', 'value_count', 'time_diffmean', 'time_diffqmean', 'time_diffmax', 'time_diffmin', 'time_diffmedian', 'time_diffstd', 'time_diffvar', 'time_burstiness', 'time_total', 'time_entropy', 'time_slope']

In [46]:
raw_train_sets = [
    pd.read_csv("../downloads/train_data_features_v3_fixed/train_features_full_v3.csv"),
    pd.read_csv("../downloads/train_data_features_v3_fixed/train_features_split1_2_v3.csv"),
    pd.read_csv("../downloads/train_data_features_v3_fixed/train_features_split2_2_v3.csv"),
    # pd.read_csv("../downloads/train_data_features_v3_fixed/train_features_split1_3_v3.csv"),
    # pd.read_csv("../downloads/train_data_features_v3_fixed/train_features_split2_3_v3.csv"),
    # pd.read_csv("../downloads/train_data_features_v3_fixed/train_features_split3_3_v3.csv"),
    # pd.read_csv("../downloads/train_data_features_v3_fixed/train_features_split1_4_v3.csv"),
    # pd.read_csv("../downloads/train_data_features_v3_fixed/train_features_split2_4_v3.csv"),
    # pd.read_csv("../downloads/train_data_features_v3_fixed/train_features_split3_4_v3.csv"),
    # pd.read_csv("../downloads/train_data_features_v3_fixed/train_features_split4_4_v3.csv")
]

In [47]:
raw_test_X = pd.read_csv("../downloads/test_features_full_v3.csv")

In [53]:
raw_train_y = pd.read_csv("../downloads/train_y_v0.1.0.csv")

In [54]:
zipftest = ZipFile('../downloads/test_X_v0.1.0.zip', 'r')
raw_listtestfile = pd.Series(zipftest.namelist()[1:]).apply(lambda x: x.split("/")[-1])

In [116]:
zs_train_sets = [trn.iloc[trn_zs_idx][zero_slope_feat_cols].reset_index(drop=True) for trn in raw_train_sets]
nz_train_sets = [trn.iloc[trn_nz_idx][pre_feature_list].reset_index(drop=True) for trn in raw_train_sets]

zs_test_X = raw_test_X.iloc[tst_zs_idx][zero_slope_feat_cols].reset_index(drop=True)
nz_test_X = raw_test_X.iloc[tst_nz_idx][pre_feature_list].reset_index(drop=True)

zs_train_y = raw_train_y.iloc[trn_zs_idx].reset_index(drop=True)
nz_train_y = raw_train_y.iloc[trn_nz_idx].reset_index(drop=True)

zs_test_filenames = raw_listtestfile[tst_zs_idx].reset_index(drop=True)
nz_test_filenames = raw_listtestfile[tst_nz_idx].reset_index(drop=True)

In [117]:
print(f"Zero slope train: {len(zs_train_sets[0])}")
print(f"Zero-Slope train label: {len(zs_train_y)}")
print(f"Zero slope test: {len(zs_test_X)}")
print(f"Zero-Slope test files: {len(zs_test_filenames)}")
print()
print(f"Non-Zero slope train: {len(nz_train_sets[0])}")
print(f"Non-Zero slope train label: {len(nz_train_y)}")
print(f"Non-Zero slope test: {len(nz_test_X)}")
print(f"Non-Zero slope test files: {len(nz_test_filenames)}")

Zero slope train: 6848
Zero-Slope train label: 6848
Zero slope test: 89408
Zero-Slope test files: 89408

Non-Zero slope train: 24991
Non-Zero slope train label: 24991
Non-Zero slope test: 226312
Non-Zero slope test files: 226312


### Data Split

In [65]:
def create_folds(train_y, n_splits=5):
    # Create a label array for stratification
    # We'll use the first non-zero label for each row as the stratification target
    stratify_labels = []
    for _, row in train_y.iterrows():
        labels = row[train_y.columns != 'filename'].values
        # Get first non-negative label, or 0 if all negative
        first_positive = next((i for i, x in enumerate(labels) if x >= 0), 0)
        stratify_labels.append(first_positive)
    
    # Create StratifiedKFold object
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # Generate fold indices
    folds = []
    for train_idx, val_idx in skf.split(train_y, stratify_labels):
        folds.append({
            'train': train_idx,
            'val': val_idx
        })
    
    return folds

In [66]:
zs_folds = create_folds(zs_train_y)
nz_folds = create_folds(nz_train_y)

# Prepare labels

In [80]:
def get_level_labels(train_y):
    level_labels = [list(train_y.columns[1:]), [], [], [], []]

    for k in range(0, 4):
        check_labels = level_labels[k]
        label_len = len(check_labels)
        idx_is_subset_of_col = pd.DataFrame(0, index=check_labels, columns=check_labels)
        is_subset = []

        for i in tqdm(range(label_len), desc=f"Processing level {k}"):
            for j in range(label_len):
                src_lb, tgt_lb = check_labels[i], check_labels[j]
                src = train_y[train_y[src_lb] == 1]
                tgt = train_y[(train_y[src_lb] == 1) & (train_y[tgt_lb] == 1)]

                idx_is_subset_of_col.loc[src_lb, tgt_lb] = len(src) <= len(tgt)
                if len(src) <= len(tgt) and src_lb != tgt_lb:
                    is_subset.append([src_lb, tgt_lb])

        remove_label = set([s[0] for s in is_subset])

        for rl in remove_label:
            level_labels[k].remove(rl)
            level_labels[k+1].append(rl)

    return level_labels

def get_processed_label(train_y, level_labels):
    def get_active_labels_np(row):
        """More efficient version using numpy"""
        arr = row.to_numpy() # convert to numpy array
        indices = np.where(arr == 1)[0] # get indices where value is 1
        labels = row.index[indices].tolist() # get labels from indices
        return labels

    labelhir = train_y.apply(get_active_labels_np, axis=1).tolist()

    tiers = {
        1: level_labels[0],
        2: level_labels[1],
        3: level_labels[2],
        4: level_labels[3],
        5: level_labels[4]
    }

    def get_tier(label):
        for tier_num, tier_list in tiers.items():
            if label in tier_list:
                return tier_num
        return None  # Handle cases where the label isn't found in any tier

    def sort_labels(labels):
        return sorted(labels, key=lambda label: (get_tier(label) or float('inf'), label))
    
    sorted_labelhir = [sort_labels(labels) for labels in labelhir]

    label_hier = np.array(
        sorted_labelhir,
        dtype=object,
    )

    padded_label = pd.Series(label_hier).apply(lambda x: x + ['None'] * (5 - len(x)) if len(x) < 5 else x)

    return padded_label


In [81]:
level_labels = get_level_labels(nz_train_y)

print(f"Preparing zero slope train labels, {len(zs_train_y)} samples")
zs_labels = get_processed_label(zs_train_y, level_labels)

print(f"Preparing non-zero slope train labels, {len(nz_train_y)} samples")
nz_labels = get_processed_label(nz_train_y, level_labels)

Processing level 3: 100%|██████████| 35/35 [00:01<00:00, 21.03it/s]


Preparing zero slope train labels, 6848 samples
Preparing non-zero slope train labels, 24991 samples


In [82]:
# Count Nones at each level
print("Zero Slope")
for i in range(5):
    none_count = sum(zs_labels.apply(lambda x: x[i] == 'None'))
    print(f"Level {i+1}: {none_count} None values out of {len(zs_labels)} total ({none_count/len(zs_labels):.2%})")

print()
print("Non Zero Slope")
# Count Nones at each level
for i in range(5):
    none_count = sum(nz_labels.apply(lambda x: x[i] == 'None'))
    print(f"Level {i+1}: {none_count} None values out of {len(nz_labels)} total ({none_count/len(nz_labels):.2%})")

Zero Slope
Level 1: 0 None values out of 6848 total (0.00%)
Level 2: 5549 None values out of 6848 total (81.03%)
Level 3: 6082 None values out of 6848 total (88.81%)
Level 4: 6730 None values out of 6848 total (98.28%)
Level 5: 6830 None values out of 6848 total (99.74%)

Non Zero Slope
Level 1: 0 None values out of 24991 total (0.00%)
Level 2: 6772 None values out of 24991 total (27.10%)
Level 3: 14165 None values out of 24991 total (56.68%)
Level 4: 20486 None values out of 24991 total (81.97%)
Level 5: 24106 None values out of 24991 total (96.46%)


# Model Training

In [111]:
from typing import List

def train_random_forest(
    train_X: List[pd.DataFrame],
    _label,
    folds,
    model_class,
    params: dict,
    none_ratio_thr: float,
    level_id: int
):
    """
    Train random forest models using k-fold cross validation
    
    Args:
        train_X: Training features DataFrame
        _label: Array of labels
        folds: List of dictionaries containing train/val indices
        
    Returns:
        tuple: (list of trained classifiers, list of scores, list of validation predictions)
    """
    classifiers = []
    val_feat_df_list = []

    for f_idx, fold in enumerate(folds):
        # Prepare train and validation data for this fold
        train_X_fold_list = []
        train_y_fold_list = []
        for trn_x in train_X:
            train_X_fold_list.append(trn_x.iloc[fold['train']])
            train_y_fold_list.append(_label[fold['train']])

        train_X_fold = pd.concat(train_X_fold_list)
        train_y_fold = np.concatenate(train_y_fold_list)

        valid_X_fold_list = []
        valid_y_fold_list = []
        for trn_x in train_X:
            valid_X_fold_list.append(trn_x.iloc[fold['val']])
            valid_y_fold_list.append(_label[fold['val']])

        val_X_fold = pd.concat(valid_X_fold_list)
        val_y_fold = np.concatenate(valid_y_fold_list)
        
        # Check the train_y_fold. If more than 30% of samples are labeled "None",
        # randomly sample from the "None" to make that ratio no more than 30%.
        none_mask = (train_y_fold == "None")
        none_count = np.sum(none_mask)
        total_samples = len(train_y_fold)
        none_ratio = none_count / total_samples if total_samples > 0 else 0

        if none_ratio > none_ratio_thr:
            # Calculate how many "None" labels we should keep (30% of total)
            max_none_to_keep = int(none_ratio_thr * (total_samples - none_count))

            # Randomly choose which "None" labels to keep
            none_indices = np.where(none_mask)[0]

            # Fix the random seed before shuffling for reproducibility
            rng = np.random.RandomState(f_idx)
            rng.shuffle(none_indices)
            
            keep_none_indices = none_indices[:max_none_to_keep]

            # Indices of all non-"None" labels
            other_indices = np.where(~none_mask)[0]

            # Combine indices to keep and then sort
            new_indices = np.concatenate([keep_none_indices, other_indices])
            new_indices = np.sort(new_indices)  # Sort so we can index the DataFrame consistently

            # Subset the training data
            train_X_fold = train_X_fold.iloc[new_indices]
            train_y_fold = train_y_fold[new_indices]

            # print(f"Sampled: none-ratio: {none_ratio}, removed: {none_count - max_none_to_keep}")

        print(f"Train size: {len(train_X_fold)}, Valid size: {len(val_X_fold)}")
        
        # Create and train Random Forest model
        model = model_class(**params)
        model.fit(train_X_fold, train_y_fold)
        
        classifiers.append(model)
        
        # Calculate score and save predictions on validation set
        val_preds = model.predict_proba(val_X_fold)
        val_pred_df = pd.DataFrame(data=val_preds, columns=model.classes_)
        val_pred_df = val_pred_df.rename(columns={col: f"{col}_{level_id}" for col in list(model.classes_)})

        val_fold_info = []
        for _f in range(len(train_X)):
            f_info = pd.DataFrame(data=fold['val'], columns=["fold_idx"])
            f_info['dataset_idx'] = _f
            val_fold_info.append(f_info)

        val_fold_idx = pd.concat(val_fold_info)
        
        val_feat_df = pd.concat([
            val_fold_idx.reset_index(drop=True),
            val_X_fold.reset_index(drop=True),
            val_pred_df,
        ], axis=1)

        val_feat_df_list.append(val_feat_df)

    return classifiers, val_feat_df_list

def setup_prev_level_prediction(predictions, fold_num, num_datasets):
    new_train_level_x = pd.concat([predictions[i] for i in range(fold_num)], ignore_index=False).sort_values(['dataset_idx', 'fold_idx'])
    return [
        new_train_level_x[new_train_level_x['dataset_idx'] == i] \
            .drop(columns=['dataset_idx', 'fold_idx']) \
            .reset_index(drop=True)
        for i in range(num_datasets)
    ]

### Train the high precision model by allowing None prediction

In [113]:
zs_classifiers = []
zs_val_predictions = []

params = {
    'n_estimators': 100,
    'random_state': 42,
    'n_jobs': 8  # Use all available cores
}

model_cls = RandomForestClassifier

none_ratio_thr_list = [0.1, 0.15, 0.35, 0.75, 0.85]

train_input = zs_train_sets

for i in range(5):
    print(f"Training level {i}")
    _classifiers, _val_predictions = train_random_forest(
        train_input,
        np.array([x[i] for x in zs_labels]),
        zs_folds,
        params=params,
        model_class=model_cls,
        none_ratio_thr=none_ratio_thr_list[i],
        level_id=i
    )
    zs_classifiers.append(_classifiers)
    zs_val_predictions.append(_val_predictions)

    train_input = setup_prev_level_prediction(_val_predictions, fold_num=len(zs_folds), num_datasets=len(train_input))
    

Training level 0
Train size: 16434, Valid size: 4110
Train size: 16434, Valid size: 4110
Train size: 16434, Valid size: 4110
Train size: 16437, Valid size: 4107
Train size: 16437, Valid size: 4107
Training level 1
Train size: 3584, Valid size: 4110
Train size: 3581, Valid size: 4110
Train size: 3584, Valid size: 4110
Train size: 3588, Valid size: 4107
Train size: 3588, Valid size: 4107
Training level 2
Train size: 2466, Valid size: 4110
Train size: 2478, Valid size: 4110
Train size: 2486, Valid size: 4110
Train size: 2486, Valid size: 4107
Train size: 2490, Valid size: 4107
Training level 3
Train size: 488, Valid size: 4110
Train size: 504, Valid size: 4110
Train size: 493, Valid size: 4110
Train size: 504, Valid size: 4107
Train size: 488, Valid size: 4107
Training level 4
Train size: 77, Valid size: 4110
Train size: 77, Valid size: 4110
Train size: 77, Valid size: 4110
Train size: 83, Valid size: 4107
Train size: 83, Valid size: 4107


In [114]:
nz_classifiers = []
nz_val_predictions = []

params = {
    'n_estimators': 100,
    'random_state': 42,
    'n_jobs': 8  # Use all available cores
}

model_cls = RandomForestClassifier

none_ratio_thr_list = [0.1, 0.15, 0.35, 0.75, 0.85]

train_input = nz_train_sets

for i in range(5):
    print(f"Training level {i}")
    _classifiers, _val_predictions = train_random_forest(
        train_input,
        np.array([x[i] for x in nz_labels]),
        nz_folds,
        params=params,
        model_class=model_cls,
        none_ratio_thr=none_ratio_thr_list[i],
        level_id=i
    )
    nz_classifiers.append(_classifiers)
    nz_val_predictions.append(_val_predictions)

    train_input = setup_prev_level_prediction(_val_predictions, fold_num=len(nz_folds), num_datasets=len(train_input))
    

Training level 0
Train size: 59976, Valid size: 14997
Train size: 59979, Valid size: 14994
Train size: 59979, Valid size: 14994
Train size: 59979, Valid size: 14994
Train size: 59979, Valid size: 14994
Training level 1
Train size: 50301, Valid size: 14997
Train size: 50273, Valid size: 14994
Train size: 50294, Valid size: 14994
Train size: 50287, Valid size: 14994
Train size: 50266, Valid size: 14994
Training level 2
Train size: 35093, Valid size: 14997
Train size: 35044, Valid size: 14994
Train size: 35129, Valid size: 14994
Train size: 35044, Valid size: 14994
Train size: 35068, Valid size: 14994
Training level 3
Train size: 18947, Valid size: 14997
Train size: 18921, Valid size: 14994
Train size: 19010, Valid size: 14994
Train size: 18768, Valid size: 14994
Train size: 18957, Valid size: 14994
Training level 4
Train size: 3901, Valid size: 14997
Train size: 3901, Valid size: 14994
Train size: 3973, Valid size: 14994
Train size: 3946, Valid size: 14994
Train size: 3923, Valid size: 1

In [118]:
zs_cliped_test_X = np.clip(zs_test_X, a_min=None, a_max=np.finfo(np.float32).max)
nz_cliped_test_X = np.clip(nz_test_X, a_min=None, a_max=np.finfo(np.float32).max)

In [891]:
# def make_predictions_with_models(classifiers, test_data):
#     """
#     Make probability predictions using multiple classifier models
    
#     Args:
#         classifiers: List of trained classifier models
#         test_data: Test data to make predictions on
        
#     Returns:
#         List of probability predictions from each classifier
#     """
#     test_preds_all = []
#     for clf in tqdm(classifiers):
#         pred = clf.predict_proba(test_data)
#         test_preds_all.append(pred)
#     return test_preds_all

In [892]:
# def align_and_combine_predictions(classifiers, test_preds_all, test_data, threshold=0.0):
#     """
#     Aligns predictions from multiple classifiers and combines them through averaging
    
#     Args:
#         classifiers: List of trained classifier models
#         test_preds_all: List of probability predictions from each classifier
#         test_data: Test data used for predictions
#         threshold: Minimum probability threshold for making predictions
        
#     Returns:
#         Final class predictions after aligning and combining probabilities
#     """
#     # Get the common classes across all classifiers
#     all_classes = classifiers[0].classes_
#     test_preds_aligned = []

#     # Make predictions with each fold's model and align them 
#     for i, clf in tqdm(enumerate(classifiers)):
#         pred = test_preds_all[i]
#         # Create a mapping to align predictions with common classes
#         pred_dict = {_cls: idx for idx, _cls in enumerate(clf.classes_)}
#         aligned_pred = np.zeros((len(test_data), len(all_classes)))
        
#         for i, _cls in enumerate(all_classes):
#             if _cls in pred_dict:
#                 aligned_pred[:, i] = pred[:, pred_dict[_cls]]
        
#         test_preds_aligned.append(aligned_pred)

#     # Stack and average the aligned predictions
#     test_preds_all = np.stack(test_preds_aligned)
#     test_preds_proba = test_preds_all.mean(axis=0)

#     # Get max probabilities for each prediction
#     max_probs = np.max(test_preds_proba, axis=1)
    
#     # Convert probabilities to class predictions, using threshold
#     test_preds = np.array(['None'] * len(test_data), dtype=object)
#     confident_mask = max_probs >= threshold
#     test_preds[confident_mask] = all_classes[np.argmax(test_preds_proba[confident_mask], axis=1)]
    
#     return test_preds

In [145]:
def make_predictions_with_models(classifiers, test_data, level_id):
    test_preds_all = []
    for clf in tqdm(classifiers):
        pred = clf.predict_proba(test_data)
        test_preds_all.append(
            pd.DataFrame(data=pred, columns=[f"{col}_{level_id}" for col in clf.classes_])
        )
    return test_preds_all

def get_pred_list(classifiers, test_input):
    test_preds_list = []
    for i in range(5):
        print(f"Predicting level {i}")
        test_preds_all = make_predictions_with_models(classifiers[i], test_input, level_id=i)

        _level_res = pd.concat(test_preds_all, axis=1).groupby(level=0, axis=1).mean()
        test_input = pd.concat([test_input, _level_res], axis=1)
        
        test_preds_list.append([df.rename(columns={col: col[:-2] for col in df.columns}) for df in test_preds_all])

    return test_preds_list

def get_test_agg(test_preds_list):
    test_level_agg = []
    for _level in tqdm(range(5)):
        _level_res = pd.concat(test_preds_list[_level], axis=1).groupby(level=0, axis=1).mean()
        assert not _level_res.isna().values.any()

        for col in _level_res.columns:
            _level_res = _level_res.rename(columns={col: f"{col}_{_level}"})

        test_level_agg.append(_level_res)

    return test_level_agg

def get_stacked_res(test_level_agg):
    stacked = np.stack(
        test_level_agg[i].idxmax(axis=1).apply(lambda x: x[:-2])
        for i in range(5)
    ).transpose()

    for row in tqdm(stacked):
        # Find first occurrence of 'None' if any
        none_idx = np.where(row == 'None')[0]
        if len(none_idx) > 0:
            # Set all elements after first None to None
            first_none = none_idx[0]
            row[first_none:] = 'None'
            
    return stacked

def post_processing(test_preds, columnlist, listtestfile):
    stackedfinalresult = pd.DataFrame(columns=['filename'])
    stackedfinalresult['filename'] = pd.Series(listtestfile).apply(lambda x: x.split("/")[-1])

    for labelname in columnlist:
        stackedfinalresult[labelname] = 0

    for i in tqdm(range(len(test_preds))):
        predlist = test_preds[i].tolist()
        predlist = [x for x in predlist if x != 'None']
        for predlabelname in predlist:
            stackedfinalresult.loc[i, predlabelname] = 1

    stackedfinalresult = stackedfinalresult.assign(**{col: stackedfinalresult[col].astype(float) for col in stackedfinalresult.columns if col != "filename"})
    
    return stackedfinalresult

def test_result_pipeline(classifiers, cliped_test_X, columnlist, listtestfile):
    test_preds_list = get_pred_list(classifiers, cliped_test_X)
    test_level_agg = get_test_agg(test_preds_list)
    test_preds = get_stacked_res(test_level_agg)
    
    return post_processing(test_preds, columnlist, listtestfile)

In [146]:
columnlist = ['Active_Power_Sensor', 'Air_Flow_Sensor',
       'Air_Flow_Setpoint', 'Air_Temperature_Sensor',
       'Air_Temperature_Setpoint', 'Alarm', 'Angle_Sensor',
       'Average_Zone_Air_Temperature_Sensor',
       'Chilled_Water_Differential_Temperature_Sensor',
       'Chilled_Water_Return_Temperature_Sensor',
       'Chilled_Water_Supply_Flow_Sensor',
       'Chilled_Water_Supply_Temperature_Sensor', 'Command',
       'Cooling_Demand_Sensor', 'Cooling_Demand_Setpoint',
       'Cooling_Supply_Air_Temperature_Deadband_Setpoint',
       'Cooling_Temperature_Setpoint', 'Current_Sensor',
       'Damper_Position_Sensor', 'Damper_Position_Setpoint', 'Demand_Sensor',
       'Dew_Point_Setpoint', 'Differential_Pressure_Sensor',
       'Differential_Pressure_Setpoint',
       'Differential_Supply_Return_Water_Temperature_Sensor',
       'Discharge_Air_Dewpoint_Sensor', 'Discharge_Air_Temperature_Sensor',
       'Discharge_Air_Temperature_Setpoint',
       'Discharge_Water_Temperature_Sensor', 'Duration_Sensor',
       'Electrical_Power_Sensor', 'Energy_Usage_Sensor',
       'Filter_Differential_Pressure_Sensor', 'Flow_Sensor', 'Flow_Setpoint',
       'Frequency_Sensor', 'Heating_Demand_Sensor', 'Heating_Demand_Setpoint',
       'Heating_Supply_Air_Temperature_Deadband_Setpoint',
       'Heating_Temperature_Setpoint', 'Hot_Water_Flow_Sensor',
       'Hot_Water_Return_Temperature_Sensor',
       'Hot_Water_Supply_Temperature_Sensor', 'Humidity_Setpoint',
       'Load_Current_Sensor', 'Low_Outside_Air_Temperature_Enable_Setpoint',
       'Max_Air_Temperature_Setpoint', 'Min_Air_Temperature_Setpoint',
       'Outside_Air_CO2_Sensor', 'Outside_Air_Enthalpy_Sensor',
       'Outside_Air_Humidity_Sensor',
       'Outside_Air_Lockout_Temperature_Setpoint',
       'Outside_Air_Temperature_Sensor', 'Outside_Air_Temperature_Setpoint',
       'Parameter', 'Peak_Power_Demand_Sensor', 'Position_Sensor',
       'Power_Sensor', 'Pressure_Sensor', 'Rain_Sensor',
       'Reactive_Power_Sensor', 'Reset_Setpoint',
       'Return_Air_Temperature_Sensor', 'Return_Water_Temperature_Sensor',
       'Room_Air_Temperature_Setpoint', 'Sensor', 'Setpoint',
       'Solar_Radiance_Sensor', 'Speed_Setpoint', 'Static_Pressure_Sensor',
       'Static_Pressure_Setpoint', 'Status', 'Supply_Air_Humidity_Sensor',
       'Supply_Air_Static_Pressure_Sensor',
       'Supply_Air_Static_Pressure_Setpoint', 'Supply_Air_Temperature_Sensor',
       'Supply_Air_Temperature_Setpoint', 'Temperature_Sensor',
       'Temperature_Setpoint', 'Thermal_Power_Sensor', 'Time_Setpoint',
       'Usage_Sensor', 'Valve_Position_Sensor', 'Voltage_Sensor',
       'Warmest_Zone_Air_Temperature_Sensor', 'Water_Flow_Sensor',
       'Water_Temperature_Sensor', 'Water_Temperature_Setpoint',
       'Wind_Direction_Sensor', 'Wind_Speed_Sensor',
       'Zone_Air_Dewpoint_Sensor', 'Zone_Air_Humidity_Sensor',
       'Zone_Air_Humidity_Setpoint', 'Zone_Air_Temperature_Sensor'
]

In [149]:
zs_test_pred_list = test_result_pipeline(zs_classifiers, zs_cliped_test_X, columnlist, zs_test_filenames)
nz_test_pred_list = test_result_pipeline(nz_classifiers, nz_cliped_test_X, columnlist, nz_test_filenames)

Predicting level 0


100%|██████████| 5/5 [00:01<00:00,  3.99it/s]


Predicting level 1


100%|██████████| 5/5 [00:02<00:00,  2.34it/s]


Predicting level 2


...

100%|██████████| 5/5 [00:10<00:00,  2.04s/it]


Predicting level 3


100%|██████████| 5/5 [00:10<00:00,  2.04s/it]


Predicting level 4


100%|██████████| 226312/226312 [00:53<00:00, 4217.83it/s]


In [152]:
final_res = pd.concat([zs_test_pred_list, nz_test_pred_list])
final_res

,filename,Active_Power_Sensor,Air_Flow_Sensor,Air_Flow_Setpoint,Air_Temperature_Sensor,Air_Temperature_Setpoint,Alarm,Angle_Sensor,Average_Zone_Air_Temperature_Sensor,Chilled_Water_Differential_Temperature_Sensor,...,Warmest_Zone_Air_Temperature_Sensor,Water_Flow_Sensor,Water_Temperature_Sensor,Water_Temperature_Setpoint,Wind_Direction_Sensor,Wind_Speed_Sensor,Zone_Air_Dewpoint_Sensor,Zone_Air_Humidity_Sensor,Zone_Air_Humidity_Setpoint,Zone_Air_Temperature_Sensor
0,test_X302336.pkl,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,test_X165301.pkl,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,test_X294340.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,test_X113297.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,test_X50858.pkl,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226307,test_X248232.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
226308,test_X196290.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
226309,test_X216428.pkl,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
226310,test_X325790.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [153]:
def check_pred_num(_final_res, thr=0.4):
    # Exclude 'filename' column if it exists
    filtered_df = _final_res.drop(columns=['filename'], errors='ignore')

    return (filtered_df >= thr).sum(axis=1)

In [154]:
"""
1    117837
2     86889
3     65924
4     34575
5     10495

1    116064
2     92604
3     62640
4     32632
5     11780
"""
check_pred_num(final_res, thr=0.35).value_counts()

2    135579
3     80988
1     52499
4     31697
5     14957
Name: count, dtype: int64

In [156]:
final_res.to_csv("../logs/submit/0125_slope_dev_5f_2aug.csv", index=False)

In [909]:
stackedfinalresult

,filename,Active_Power_Sensor,Air_Flow_Sensor,Air_Flow_Setpoint,Air_Temperature_Sensor,Air_Temperature_Setpoint,Alarm,Angle_Sensor,Average_Zone_Air_Temperature_Sensor,Chilled_Water_Differential_Temperature_Sensor,...,Warmest_Zone_Air_Temperature_Sensor,Water_Flow_Sensor,Water_Temperature_Sensor,Water_Temperature_Setpoint,Wind_Direction_Sensor,Wind_Speed_Sensor,Zone_Air_Dewpoint_Sensor,Zone_Air_Humidity_Sensor,Zone_Air_Humidity_Setpoint,Zone_Air_Temperature_Sensor
0,test_X20367.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,test_X103084.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,test_X6910.pkl,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,test_X66332.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,test_X38528.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315715,test_X325790.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315716,test_X61444.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315717,test_X221284.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315718,test_X115827.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
import numpy as np

pred_idx = np.load("../logs/submit/0124_semi_supervise_balanced.npy")

In [31]:
pred_res = np.zeros((len(listtestfile), len(columnlist)))
pred_res[pred_idx[0], pred_idx[1]] = 1.0

In [32]:
semi_pred_filename = pd.DataFrame(columns=['filename'])
semi_pred_filename['filename'] = pd.Series(listtestfile).apply(lambda x: x.split("/")[-1])

semi_pred_res = pd.DataFrame(data=pred_res, columns=columnlist)

semi_pred_res = pd.concat([semi_pred_filename, semi_pred_res], axis=1)

In [33]:
semi_pred_res.to_csv("../logs/submit/0124_semi_super_rf_balanced.csv", index=False)

In [34]:
semi_pred_res

,filename,Active_Power_Sensor,Air_Flow_Sensor,Air_Flow_Setpoint,Air_Temperature_Sensor,Air_Temperature_Setpoint,Alarm,Angle_Sensor,Average_Zone_Air_Temperature_Sensor,Chilled_Water_Differential_Temperature_Sensor,...,Warmest_Zone_Air_Temperature_Sensor,Water_Flow_Sensor,Water_Temperature_Sensor,Water_Temperature_Setpoint,Wind_Direction_Sensor,Wind_Speed_Sensor,Zone_Air_Dewpoint_Sensor,Zone_Air_Humidity_Sensor,Zone_Air_Humidity_Setpoint,Zone_Air_Temperature_Sensor
0,test_X20367.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,test_X103084.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,test_X6910.pkl,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,test_X66332.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,test_X38528.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315715,test_X325790.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315716,test_X61444.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315717,test_X221284.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315718,test_X115827.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
